# Lab 3, Part 1 : N_Gram on WikiText

## Download WikiText-2 dataset

In [1]:
#wget is not working. urlretrieve is a good alternative
"""
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt
! wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/test.txt
"""

import urllib.request
from tqdm import tqdm
import os
from urllib.parse import urlparse

urls = [
        "https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt",
        "https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt",
        "https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/test.txt"
        ]

"""
Shows a progress bar for the task at hand(urlretrieve)
@:param unit: progression Unit. Set to Bytes
@:param unit_scale: number of iteration. Set to be scaled automatically
@:param Minimum progress display update interval in iterations. Set to 1 in case download speed is erratic
@:param Prefix of the progress bar
"""
""""""
for url in urls:
    a = urlparse(url)
    with tqdm(unit='B', unit_scale=True, miniters=1, desc=os.path.basename(a.path)) as progressBar: # getting a progress bar because I wasn't getting one when using the urlretrieve command
       urllib.request.urlretrieve(url, os.path.basename(a.path), reporthook=lambda blocks, block_size, total_size: progressBar.update(block_size))



train.txt: 10.8MB [00:00, 20.9MB/s]
valid.txt: 1.13MB [00:00, 11.5MB/s]
test.txt: 1.27MB [00:00, 14.4MB/s]


## Helper function

In [3]:
from collections import Counter, defaultdict
import math
import copy
import random
import operator

flatten = lambda l: [item for sublist in l for item in sublist]

"""
Does the necessary preprocessing for the dataset
@:param filename: name of the file to be preprocessed
@:return vocab: set of all the words in the corpus
"""
def prepare_data(filename):
    data = [l.strip().lower().split() + ['</s>'] for l in open(filename, encoding='utf-8') if l.strip()] # adding </s> at the end of each sentence
    corpus = flatten(data) # flattening the list, so that we have a list of words, instead of a list of sentences
    vocab = set(corpus) # getting the vocabulary, which is the set of all the words in the corpus, set() removes duplicates
    return vocab, data

# About this lab

In this notebook, you will work with the N-Gram Language Model on the Wiki Text Dataset. This first lab should take you approximatively 1h.

<div align ="center"><img src="https://www.w3.org/TR/ngram-spec/tree2.gif"></div>

# A - The N-Gram Language Model

A language model assigns a probability to each possible next word given a history of previous words (context). 

<div align='center'> $P(w_t|w_{t-1}, w_{t-2}, ... , w_1)$ </div>



## Markov Assumption

Since calculating the probability of the whole sentence is not feasible, the Markov Assumption is introduced.

It assumes that each next word only depends on the previous K words (in an N-Gram language model, K = N-1).
- Unigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t)$
- Bigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t|w_{t-1}) $
- Trigram: $P(w_t|w_{t-1}, w_{t-2}, ... , w_1) = P(w_t|w_{t-1}, w_{t-2})$

For an N-Gram language model, the probability is calculated by counting the frequency:

$P(w_t|w_{t-1}, w_{t-2}, ... ,w_{t-N+1}) = \frac{C(w_t, w_{t-1}, w_{t-2}, ... ,w_{t-N+1} )}{C(w_{t-1}, w_{t-2}, ... ,w_{t-N+1})}$


We provide you the code of the NGram Language Model.

* Understand the code

In [26]:
"""
This class implements the N-Gram Language Model
"""
class NGramLM():
    def __init__(self, N):
        self.N = N
        self.vocab = set()
        self.data = []
        self.prob = {}
        self.counts = defaultdict(Counter)
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word]
    # For N > 1, the probability is in a nested dict   P = prob[context][next_word]
    """
    Calculates the probability of the next word given the context
    @:param context: list of words
    @:param word: next word
    @:return: probability of the next word given the context
    """
    def train(self, vocab, data, smoothing_k=0):
        self.vocab = vocab
        self.data = data
        self.smoothing_k = smoothing_k # user te eliminate noise from the dataset

        if self.N == 1:
            self.counts = Counter(flatten(data))
            self.prob = self.get_prob(self.counts)
        else:
            self.vocab.add('<s>')
            counts = self.count_ngram()
            
            self.prob = {}
            for context, counter in counts.items():
                self.prob[context] = self.get_prob(counter)
    """
    Calculates the probability of the next word given the context
    @:param context: list of words
    @:param word: next word
    @:return: probability of the next word given the context
    """
    def count_ngram(self):
        counts = defaultdict(Counter)
        for sentence in self.data:
            sentence = (self.N - 1) * ['<s>'] + sentence 
            for i in range(len(sentence)-self.N+1):
                context = sentence[i:i+self.N-1]
                context = " ".join(context)
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word
                word = sentence[i+self.N-1]
                counts[context][word] += 1

        self.counts = counts
        return counts
        
    # normalize counts into probability(considering smoothing)
    """
    Calculates the probability of the next word given the context
    @:param context: list of words
    @:param word: next word
    @:return: probability of the next word given the context
    """
    def get_prob(self, counter):
        total = float(sum(counter.values()))
        k = self.smoothing_k
    
    # For N = 1, the probability is stored in a dict   P = prob[next_word
        
        prob = {}
        for word, count in counter.items():
            prob[word] = (count + k) / (total + len(self.vocab) * k)
        return prob
    """
    Calculates the probability of the next word given the context
    @:param context: list of words
    @:param word: next word
    @:return: probability of the next word given the context
    """
    def get_ngram_logprob(self, word, seq_len, context=""):
        if self.N == 1 and word in self.prob.keys():
            return math.log(self.prob[word]) / seq_len
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return math.log(self.prob[context][word]) / seq_len
        else:
            # assign a small probability to the unseen ngram
            # to avoid log of zero and to penalise unseen word or context
            return math.log(1/len(self.vocab)) / seq_len
    """
    Calculates the probability of the next word given the context
    @:param context: list of words
    @:param word: next word
    @:return: probability of the next word given the context
    """
    def get_ngram_prob(self, word, context=""):
        if self.N == 1 and word in self.prob.keys():
            return self.prob[word]
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return self.prob[context][word]
        elif word in self.vocab and self.smoothing_k > 0:
            # probability assigned by smoothing
            return self.smoothing_k / (sum(self.counts[context].values()) + self.smoothing_k*len(self.vocab))
        else:
            # unseen word or context
            return 0
            
    # In this method, the perplexity is measured at the sentence-level, averaging over all sentences.
    # Actually, it is also possible to calculate perplexity by merging all sentences into a long one.
    """
    Calculates the perplexity of the test data
    @:param test_data: list of sentences
    @:return: perplexity of the test data
    """
    def perplexity(self, test_data):
        log_ppl = 0
        if self.N == 1:
            for sentence in test_data:
                for word in sentence:
                    log_ppl += self.get_ngram_logprob(word=word, seq_len=len(sentence))
        else:
            for sentence in test_data:
                for i in range(len(sentence)-self.N+1):
                    context = sentence[i:i+self.N-1]
                    context = " ".join(context)
                    word = sentence[i+self.N-1]
                    log_ppl += self.get_ngram_logprob(context=context, word=word, seq_len=len(sentence))
        log_ppl /= len(test_data)
        ppl = math.exp(-log_ppl)
        return ppl
    """
    Calculates the perplexity of the test data
    @:param test_data: list of sentences
    @:return: perplexity of the test data
    """
    def _is_unseen_ngram(self, context, word):
        if context not in self.prob.keys() or word not in self.prob[context].keys():
            return True
        else:
            return False
    
    # generate the most probable k words
    """
    Calculates the perplexity of the test data
    @:param test_data: list of sentences
    @:return: perplexity of the test data
    """
    def generate_next(self, context, k):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        if ngram_context in self.prob.keys():
            candidates = self.prob[ngram_context]
            most_probable_words = sorted(candidates.items(), key=lambda kv: kv[1], reverse=True)
            for i in range(min(k, len(most_probable_words))):
                print(" ".join(context[self.N-1:])+" "+most_probable_words[i][0]+"\t P={}".format(most_probable_words[i][1]))
        else:
            print("Unseen context!")
            
    # generate the next n words with greedy search
    """
    Calculates the perplexity of the test data
    @:param test_data: list of sentences
    @:return: perplexity of the test data
    """
    def generate_next_n(self, context, n):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)
        
        for i in range(n):
            try:
                candidates = self.prob[ngram_context]
                most_likely_next = max(candidates.items(), key=operator.itemgetter(1))[0]
                context += [most_likely_next]
                ngram_context_list = ngram_context_list[1:] + [most_likely_next]
                ngram_context = " ".join(ngram_context_list)
            except:
                break
        print(" ".join(context[self.N-1:]))
    

# B -Train with toy dataset

As the model is created we can train a N-Gram LM. At this step, let's train a Bigram language model on the toy dataset.

* What preprocessing was done ?
* How many word do we have in our vocabulary ?

strip  : take off spaces at the beginning and at the end of the text
lower  : text to lowercase
split  : split text on spaces
flatten: delete vocab duplicates

We have 8 words in the vocab

In [5]:
corpus = ["I like whipped cream",
         "I like candys",
         "I hate tomatoes"]
data = [l.strip().lower().split() + ['</s>'] for l in corpus if l.strip()]
vocab = set(flatten(data))

# TODO : Answer the questions
print(data)
print(vocab, len(vocab))

[['i', 'like', 'whipped', 'cream', '</s>'], ['i', 'like', 'candys', '</s>'], ['i', 'hate', 'tomatoes', '</s>']]
{'candys', 'like', '</s>', 'hate', 'cream', 'whipped', 'i', 'tomatoes'} 8


In [6]:
"""
Train a N-Gram language model
@:param data: list of sentences
"""
def print_probability(lm):
    for context in lm.vocab:
        for word in lm.vocab:
            prob = lm.get_ngram_prob(word, context)
            print("P({}\t|{}) = {}".format(word, context, prob))
        print("--------------------------")

# C - Smoothing
Smoothing method is used to deal with the sparsity problem in N-Gram language modelling.
The probability mass is shifted towards the less frequent words.

For example, with an add-1 smoothing, the probability is calculated as:

$$P(w_t | context) = \frac{C(w_t, context)+1}{C(context)+|V|}$$



**Q1: What is the disadvantage of smoothing?**

A: Given how it assigns probability to all event so as to foresee unseen events and ideally make the model more robust, it could even assign a probability to irrelevant events thus introducing noise in the model which ultimately leads to less accurate predictions
It is also ought to increase the complexity of the model if the smoothing technique is resources-consuming

In [7]:
lm = NGramLM(2)
lm.train(vocab, data, smoothing_k=0)

########################################################
# TODO: try with add-2 smoothing and see the probability
########################################################
lm.train(vocab, data, smoothing_k=2)

print_probability(lm)

P(candys	|candys) = 0.10526315789473684
P(like	|candys) = 0.10526315789473684
P(</s>	|candys) = 0.15789473684210525
P(hate	|candys) = 0.10526315789473684
P(<s>	|candys) = 0.10526315789473684
P(cream	|candys) = 0.10526315789473684
P(whipped	|candys) = 0.10526315789473684
P(i	|candys) = 0.10526315789473684
P(tomatoes	|candys) = 0.10526315789473684
--------------------------
P(candys	|like) = 0.15
P(like	|like) = 0.1
P(</s>	|like) = 0.1
P(hate	|like) = 0.1
P(<s>	|like) = 0.1
P(cream	|like) = 0.1
P(whipped	|like) = 0.15
P(i	|like) = 0.1
P(tomatoes	|like) = 0.1
--------------------------
P(candys	|</s>) = 0.1111111111111111
P(like	|</s>) = 0.1111111111111111
P(</s>	|</s>) = 0.1111111111111111
P(hate	|</s>) = 0.1111111111111111
P(<s>	|</s>) = 0.1111111111111111
P(cream	|</s>) = 0.1111111111111111
P(whipped	|</s>) = 0.1111111111111111
P(i	|</s>) = 0.1111111111111111
P(tomatoes	|</s>) = 0.1111111111111111
--------------------------
P(candys	|hate) = 0.10526315789473684
P(like	|hate) = 0.105263

# D -Train on WikiText-2 dataset and Evaluate Perplexity


## Evaluating perplexity

We define the perplexity as 

<div align="center">
$ PPL(W) = P(w_1, w_2, ... , w_n)^{-\frac{1}{n}}$

$ log(PPL(W)) = -\frac{1}{n}\sum^n_{k=1}log(P(w_k|w_1, w_2, ... , w_{k-1}))$
</div>





**Q2: Why do we need to calculate log perplexity?**

A: It helps determine if the language is overfitting or underfitting. If it is much lower that on the tes set, it may indicate the model is overfitting to the training data. On the other hand it is it too high, this indicates that the model is underfitting

In [9]:
vocab, train_data = prepare_data('train.txt')
v1, valid_data = prepare_data('valid.txt')
v2, test_data = prepare_data('test.txt')
print(vocab, len(vocab))

{'767', 'foliage', 'blotched', 'consumes', 'shampain', 'purity', 'plight', 'overrunning', 'mistletoe', 'unnumbered', 'ptv', 'tips', 'purportedly', 'straining', 'uttar', 'hummelfjell', 'beginnings', '775', '20th', 'teachers', 'butch', 'literally', 'pains', 'turnbull', 'resembles', 'catcher', 'corral', 'libraries', 'disposition', 'delegation', 'eritrea', 'southbound', 'indoors', 'realised', 'regimen', 'consulted', 'motive', 'groups', 'tempered', 'weaker', 'pettit', 'façades', 'overseer', 'vinegar', 'pinner', 'bad', 'mess', 'pollution', 'sparrow', 'childhood', 'linguist', 'northward', 'zombie', 'petition', 'unchecked', 'india', 'costume', 'hetherington', '1743', 'reala', 'barrowman', 'repute', 'imposition', 'streak', 'keyes', 'feather', 'manfred', 'tombs', 'abolished', 'kurfürst', 'registry', 'unprofitable', 'korda', 'errant', 'capablanca', 'katz', 'megastore', 'suggesting', 'neighbors', 'afterward', 'gradual', 'fittoni', 'neotrygon', 'hierarchy', 'finland', 'nucleolar', 'housewives', 'yu

In [10]:
lm = NGramLM(3)

# Train the model using vocab and training data

lm.train(vocab,train_data)

Perplexity would be in this case quantitaitve measure of how well the model is executing the task at hand. With n getting bigger it is obvious that its value range inf will be proportiontely big simply because there are more data to deal with and by extension more possibilities. Test and valid perplexities'values infer if the model is overfitting or not.A train perplexity way higher than that of test translates as the model has overfit the train data.

In [11]:
print(lm.perplexity(valid_data))
print(lm.perplexity(test_data))

387.1190374834364
310.6880368161233


# E - Generating sentences

With a pre-trained N-Gram language model, we can predict possible next words given context.

In [14]:
# generate the most probable following words given the context
print(" ".join(valid_data[12]))

# actually the only useful context in the Trigram language model is ["where", "they"]
context = "the eggs hatch at night , and the larvae swim to the water surface where they"  
lm.generate_next(context, 3)


the eggs hatch at night , and the larvae swim to the water surface where they drift with the ocean currents , preying on <unk> . this stage involves three <unk> and lasts for 15 – 35 days . after the third moult , the juvenile takes on a form closer to the adult , and adopts a <unk> lifestyle . the juveniles are rarely seen in the wild , and are poorly known , although they are known to be capable of digging extensive burrows . it is estimated that only 1 larva in every 20 @,@ 000 survives to the <unk> phase . when they reach a carapace length of 15 mm ( 0 @.@ 59 in ) , the juveniles leave their burrows and start their adult lives . </s>
the eggs hatch at night , and the larvae swim to the water surface where they were	 P=0.12408759124087591
the eggs hatch at night , and the larvae swim to the water surface where they are	 P=0.08029197080291971
the eggs hatch at night , and the larvae swim to the water surface where they would	 P=0.051094890510948905


In [15]:
# we can also generate with shorter contexts, even shorter than N-1

contexts = ["the eggs",
            "the",
            ""]
for context in contexts:
  lm.generate_next(context, 3)
  print(lm)

the eggs hatch	 P=0.18181818181818182
the eggs of	 P=0.18181818181818182
the eggs are	 P=0.13636363636363635
the first	 P=0.03398926654740608
the <unk>	 P=0.020274299344066785
the episode	 P=0.015802027429934407
 =	 P=0.26132873311734756
 the	 P=0.14112004039214035
 in	 P=0.06353347077881095


In [16]:
context = "the eggs hatch at night , and the larvae swim to the water surface where they"  

# generate the next n words with greedy search
lm.generate_next_n(context, 10)

# This is not a good method in practice,
# because wrong predictions in the early steps would introduce errors to the following predictions
lm.generate_next_n(context, 20)

the eggs hatch at night , and the larvae swim to the water surface where they were not able to get the part of the <unk>
the eggs hatch at night , and the larvae swim to the water surface where they were not able to get the part of the <unk> of the <unk> of the <unk> of the <unk> of


# F - Effect of N

**Q3: Why does the perplexity increase when N is large?**

A: The perplexity increses because there are more data to deal with and by extension more possibilities. Test and valid perplexities'values infer if the model is overfitting or not.A train perplexity way higher than that of test translates as the model has overfit the train data.

In [22]:
for n in range(1,6):
    lm = NGramLM(n)
    lm.train(vocab, train_data)
    print("************************")
    print("{}-gram LM perplexity on valid set: {}".format(n, lm.perplexity(valid_data)))
    print("{}-gram LM perplexity on test  set: {}".format(n, lm.perplexity(test_data)))

************************
1-gram LM perplexity on valid set: 599.6450225274759
1-gram LM perplexity on test  set: 553.0467265986567
************************
2-gram LM perplexity on valid set: 130.75949726991627
2-gram LM perplexity on test  set: 114.76592406312065
************************
3-gram LM perplexity on valid set: 387.1190374834364
3-gram LM perplexity on test  set: 310.6880368161233
************************
4-gram LM perplexity on valid set: 1091.9348578736633
4-gram LM perplexity on test  set: 846.7563102382584
************************
5-gram LM perplexity on valid set: 1558.823545905558
5-gram LM perplexity on test  set: 1201.746345143052


# G - Interpolation
In interpolation, we mix the probability estimates from all the n-gram estimators to alleviate the sparsity problem.

In [39]:
"""
class that implements the interpolation language model, which is a weighted sum of n-gram language models
"""
class InterpolateNGramLM(NGramLM):
"""
initialize the model
@:param N: the maximum n-gram order
"""
    def __init__(self, N):
        super(InterpolateNGramLM, self).__init__(N)
        self.ngram_lms = []
        self.lambdas = []
"""
train the model
@:param vocab: the vocabulary
@:param data: list of sentences
@:param smoothing_k: the smoothing parameter
@:param lambdas: the weights of the n-gram language models
"""
    def train(self, vocab, data, smoothing_k=0, lambdas=[]):
        assert len(lambdas) == self.N
        assert sum(lambdas) - 1 < 1e-9
        self.vocab = vocab
        self.lambdas = lambdas
        
        for i in range(self.N, 0, -1):
            # Instanciate the model
            lm = NGramLM(i)
            print("Training {}-gram language model".format(i))
            lm.train(vocab, data, smoothing_k)
            self.ngram_lms.append(lm)
"""
get the probability of a word given the context
@:param word: the word
@:param context: the context
@:param seq_len: the length of the sequence
@:return: the probability of the word given the context"""
    def get_ngram_logprob(self, word, seq_len, context):
        prob = 0
        for i, (coef, lm) in enumerate(zip(self.lambdas, self.ngram_lms)):
            context_words = context.split()
            cutted_context = " ".join(context_words[-self.N + i + 1:])
            prob += coef * lm.get_ngram_prob(context=cutted_context, word=word)
        return math.log(prob) / seq_len
        

In [40]:
###################################################
# Q5: tune your coefficients to decrease perplexity
###################################################
ilm = InterpolateNGramLM(3)
ilm.train(vocab, train_data, lambdas=[0.5, 0.4, 0.1])

Training 3-gram language model
Training 2-gram language model
Training 1-gram language model


In [43]:
print("valid: ",ilm.perplexity(valid_data))
print("train: ",ilm.perplexity(test_data))

valid:  126.26886448793077
train:  103.0773276220259
